In [13]:
import sys
print(sys.prefix)

/Users/sabkx/venv-metal


In [3]:
import torch
from torchtext.datasets import AG_NEWS

train_iter = iter(AG_NEWS(split="train"))

In [4]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [5]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
train_iter = AG_NEWS(split="train")


def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

/Users/sabkx/venv-metal/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/sabkx/venv-metal/lib/python3.12/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [6]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [26]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def collate_batch(batch):
    label_list, text_list = [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text))
        processed_text = F.pad(processed_text, (0, 50-processed_text.size(0)))
        text_list.append(processed_text)
    return torch.tensor(label_list), torch.stack(text_list)

train_iter = AG_NEWS(split="train")
dataloader = DataLoader(
    train_iter, batch_size=64, shuffle=False, collate_fn=collate_batch
)

Thanks chatGPT for the code :DDDD

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.hidden_dim = hidden_dim
        
        # Define the weights for query, key, and value matrices
        self.W_q = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_k = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_v = nn.Linear(hidden_dim, hidden_dim, bias=False)
    
    def forward(self, x):
        # x shape: (batch_size, seq_len, embedding_dim)

        # Compute queries, keys, and values
        Q = self.W_q(x)  # Shape: (batch_size, seq_len, embedding_dim)
        K = self.W_k(x)  # Shape: (batch_size, seq_len, embedding_dim)
        V = self.W_v(x)  # Shape: (batch_size, seq_len, embedding_dim)
        
        # Compute attention scores
        # Q @ K.T: (batch_size, seq_len, hidden_dim) @ (batch_size, hidden_dim, seq_len)
        # Output shape: (batch_size, seq_len, seq_len)
        attention_scores = torch.bmm(Q, K.transpose(1, 2)) / (self.hidden_dim ** 0.5)
        
        # Apply softmax to get the attention weights
        attention_weights = F.softmax(attention_scores, dim=-1)  # Shape: (batch_size, seq_len, seq_len)
        
        # Compute the weighted sum of values
        # attention_weights @ V: (batch_size, seq_len, seq_len) @ (batch_size, seq_len, hidden_dim)
        # Output shape: (batch_size, seq_len, hidden_dim)
        attention_output = torch.bmm(attention_weights, V)
        
        return attention_output

# Example usage
batch_size = 64
seq_len = 50
embedding_dim = 128

# Random input tensor
x = torch.randn(batch_size, seq_len, embedding_dim)

# Instantiate and apply self-attention
self_attention = SelfAttention(embedding_dim)
output = self_attention(x)

print("Input:", x)
print("Self-attention output:", output)


Input: tensor([[[-4.7706e-01, -1.1555e-01, -2.7620e-04,  ...,  1.5167e+00,
           1.0012e+00,  1.7228e+00],
         [ 1.7881e-01, -8.2281e-01, -6.0000e-01,  ...,  5.5161e-01,
          -2.1384e-01,  2.5889e+00],
         [-8.3890e-01,  2.9913e-01,  4.5329e-01,  ...,  3.8309e-03,
          -4.3858e-02,  1.1039e+00],
         ...,
         [ 1.6570e-01, -1.1907e+00, -1.4193e+00,  ...,  4.6234e-01,
          -1.1105e-01,  2.1113e-01],
         [-1.0796e+00,  4.1052e-01, -9.9326e-01,  ..., -1.0755e+00,
           1.1072e+00,  1.6945e-02],
         [ 6.6218e-01, -7.2390e-01,  9.8125e-01,  ...,  7.3946e-01,
          -1.4586e-01, -2.0322e-01]],

        [[-9.2782e-01,  5.2059e-02, -6.5602e-01,  ..., -9.2437e-01,
          -1.4547e+00,  5.2213e-01],
         [ 1.3634e+00, -5.7781e-01, -9.7092e-01,  ..., -3.7101e-01,
           9.3096e-01,  4.7224e-01],
         [-8.2703e-01, -1.0037e+00,  2.2094e+00,  ..., -2.2256e-01,
          -4.2365e-01,  4.2937e-01],
         ...,
         [ 2.0182e

In [67]:
class LSTM_with_Attention(nn.Module):
    def __init__(self, vocab_size, hidden_dim, embedding_dim, num_classes):
        super(LSTM_with_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm_1 = nn.LSTM(input_size=embedding_dim, 
                              hidden_size=hidden_dim,
                              num_layers=1,
                              bidirectional=False,
                              dropout=0,
                              batch_first=True)
        self.attention_1 = SelfAttention(hidden_dim=hidden_dim)
        self.lstm_2 = nn.LSTM(input_size=hidden_dim, 
                              hidden_size=hidden_dim,
                              num_layers=1,
                              bidirectional=False,
                              dropout=0,
                              batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        # print("x:", x.shape)
        embedded = self.embedding(x)
        # print(embedded.shape)
        output, (ht, ct) = self.lstm_1(embedded)
        # print("output:", output.shape)
        # print("ht[-1]:", ht[-1].shape)
        # print("ct:", ct.shape)
        # print("self.fc(ht[-1]):", self.fc(ht[-1]).shape)
        attention_output = self.attention_1(output)
        # print("attention_output:", attention_output.shape)
        output2, (ht2, ct2) = self.lstm_2(attention_output)
        # print(ht2[-1].shape)
        return self.fc(ht2[-1])


In [89]:
train_iter = AG_NEWS(split="train")
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = LSTM_with_Attention(vocab_size, 128, emsize, num_class)

In [90]:
import time


def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [91]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10  # epoch
LR = 5 # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.252
| epoch   1 |  1000/ 1782 batches | accuracy    0.256
| epoch   1 |  1500/ 1782 batches | accuracy    0.254


KeyboardInterrupt: 